In [3]:
%%HTML
### Cambiamos los tamaños de fuente asociados a las tablas
<style>
    tr { font-size: 18px; background-color: lightblue; font-color: orange}
    td { font-size: 18px; }
</style>

# [SPADE](https://spade-mas.readthedocs.io/en/latest/index.html): Smart Python Agent Development Environment 

## Instalar SPADE

In [6]:
try:
    import spade
except:
    !pip install spade==4.0.2

***
&#9654; Abre una terminal desde el entorno anaconda (primera opción del menú contextual que usaste para abrir **Jupiter**), y en la terminal lanza a ejecución **pyjabber**:

`$ pyjabber `
***

## Ejemplo 1: Dummy Agent - `dummyagent.py`

Los agentes SPADE usan comunicación mediante un servidor XMPP. Para ello, tienen que, igual que cualquier usuario, registrarse en el servidor mediante un jid y una clave.

El JID contiene el nombre del agente (antes de la @) y el DNS o IP del servidor XMPP (después de la @).

<div class="alert alert-block alert-danger">
<b>NOTA: MUY IMPORTANTE:</b> 

Un agente se distingue del resto por su identificador, por lo que debe ser único. Antes de ejecutar cualquier ejemplo o agente, cambia su identificador para que, al menos, incluya tu login y así pueda identificarse del resto. 
    
Ejemplo: En el ejemplo siguiente, el agente a ejecutar se llama `Prueba_SPADE@gtirouter.dsic.upv.es`. En mi caso, le cambiaría el nombre para que se llamase `Prueba_SPADE_carrasco@localhost` (además, como he puesto en marcha el servidor XMPP local `pyjabber`, cambiamos la direccion del servidor para que sea `localhost`).  
</div>

In [7]:
import spade

class DummyAgent(spade.agent.Agent):
    async def setup(self):
        print("Hello World! I'm agent {}".format(str(self.jid)))

### Añadid vuestro login como parte del JID del agente
dummy = DummyAgent("Prueba_SPADE_brivaro@gtirouter.dsic.upv.es", "SPADE")
await dummy.start()
await dummy.stop()
print('Prueba')
print(dummy.is_alive())

Hello World! I'm agent prueba_spade_brivaro@gtirouter.dsic.upv.es
Prueba
False


In [8]:
import spade

class DummyAgent(spade.agent.Agent):
    async def setup(self):
        print("Hello World! I'm agent {}".format(str(self.jid)))

### Añadid vuestro login como parte del JID del agente
dummy = DummyAgent("Prueba_SPADE_brivaro@gtirouter.dsic.upv.es", "SPADE")
await dummy.start()
await dummy.stop()
print('Prueba')
print(dummy.is_alive())

Hello World! I'm agent prueba_spade_brivaro@gtirouter.dsic.upv.es
Prueba
False


<div class="alert alert-block alert-info">
<b>NOTA:</b> 

Para ejecutar el programa desde una terminal fuera del notebook, en la carpeta donde tenéis el fichero `dummyagent.py` haced: `$ python dummyagent.py`
</div>

Un agente SPADE es un agente asíncrono. Esto significa que todo el código para correr un agente debe ejecutarse en un bucle asíncrono. Este bucle, denominado bucle de eventos (event loop) es el núcleo de cualquier aplicación asíncrona. Los bucles de eventos ejecutan asíncronamente tareas y callbacks, realizan operaciones de E/S y ejecutan subprocesos. 



### [Async IO](https://realpython.com/async-io-python/)

Async IO es un paradigma (modelo) de diseño de programación concurrente que tiene implementaciones en una gran cantidad de lenguajes de programación, entre ellos Python.

 - `asyncio` : el paquete de Python que proporciona una base y una API para ejecutar y gestionar corutinas
 - `async` / `await` : dos nuevas palabras clave de Python que se utilizan para definir corutinas.
     - `async def` introduce una corutina nativa. Puede utilizar `await`, `return` o `yield`, pero todos ellos son opcionales. Declarar `async def noop(): pass` es válido:
         - El uso de `await` y/o `return` crea una función corutina. Para llamar a una función corutina, debes esperarla (`await`) para obtener sus resultados.
         - Es menos común usar `yield` en un bloque `async def`. Esto crea un generador asíncrono, sobre el que iteras con `async for`.
         - Cualquier cosa definida con `async def` no puede usar `yield from`, lo que lanzará un `SyntaxError`.
     - Al igual que es un `SyntaxError` usar `yield` fuera de una función `def`, es un `SyntaxError` usar `await` fuera de una corutina `async def`. Sólo puedes usar `await` en el cuerpo de las corutinas.


     - `await` devuelve el control de la función al bucle de eventos. (Suspende la ejecución de la corutina circundante). Si Python encuentra una expresión `await f()` en el ámbito de `g()`, así es como `await` le dice al bucle de eventos, "Suspende la ejecución de `g()` hasta que lo que estoy esperando -el resultado de `f()`- sea devuelto. Mientras tanto, deja que se ejecute otra cosa".



`asyncio` es una biblioteca para escribir código concurrente. Sin embargo, async IO no es *threading*, ni multiprocesamiento. No se basa en ninguno de ellos.

De hecho, `async IO` es un diseño de un solo hilo y un solo proceso: usa **multitarea cooperativa**. Es decir, `async IO` da una sensación de concurrencia a pesar de usar un único hilo en un único proceso. Las corutinas (una característica central de `async IO`) pueden programarse concurrentemente, pero no son inherentemente concurrentes.

Para reiterar, `async IO` es un estilo de programación concurrente, pero no es paralelismo. 

Las rutinas asíncronas son capaces de "hacer una pausa" mientras esperan su resultado final y dejar que otras rutinas se ejecuten mientras tanto.
El código asíncrono, a través del mecanismo anterior, facilita la ejecución concurrente. Dicho de otro modo, el código asíncrono da la apariencia de concurrencia.

#### Ejemplo de funcionamiento asíncrono.

La maestra de ajedrez Judit Polgár organiza una exhibición de ajedrez en la que se enfrenta a varios jugadores aficionados. Tiene dos formas de llevar a cabo la exhibición: de forma sincrónica y de forma asincrónica.
        
Supuestos:
        
- 24 adversarios
- Judit realiza cada movimiento de ajedrez en 5 segundos
- Cada oponente tarda 55 segundos en realizar un movimiento.
- Las partidas tienen una media de 30 movimientos por pareja (60 movimientos en total)
        
**Versión sincrónica** : Judit juega una partida cada vez, nunca dos al mismo tiempo, hasta completar la partida. Cada partida dura (55 + 5) * 30 == 1800 segundos, es decir, 30 minutos. La exhibición completa dura 24 * 30 == 720 minutos, es decir, 12 horas.
        
**Versión asíncrona** : Judit se mueve de mesa en mesa, haciendo un movimiento en cada mesa. Abandona la mesa y deja que el oponente haga su siguiente movimiento durante el tiempo de espera. Un movimiento en las 24 partidas le lleva a Judit 24 * 5 == 120 segundos, o 2 minutos. La exhibición completa se reduce ahora a 120 * 30 == 3600 segundos, es decir, sólo 1 hora.

Así pues, la multitarea cooperativa es una forma elegante de decir que el bucle de eventos de un programa se comunica con múltiples tareas para permitir que cada una de ellas se ejecute por turnos en el momento óptimo.

La IO asíncrona se encarga de los largos periodos de espera en los que las funciones estarían bloqueándose y permite que otras funciones se ejecuten durante ese tiempo de inactividad. (Una función que se bloquea efectivamente prohíbe que otras se ejecuten desde el momento en que se inicia hasta el momento en que regresa).

Asyncio proporciona a las tareas la capacidad de ceder explícitamente el control bajo demanda durmiendo mediante `asyncio.sleep()`.

Se denomina "no bloqueante", lo que significa que no bloquea la ejecución del subproceso actual. Esto significa que mientras una coroutina o tarea está durmiendo, otras tareas y coroutinas pueden ejecutarse.

También podemos dormir durante cero segundos.

Se trata de un periodo de reposo especial. Significa que la tarea o coroutina actual se suspenderá y el bucle de eventos de asyncio permitirá que otras tareas o coroutinas se ejecuten.

La coroutina o tarea actual se reanudará tan pronto como sea posible, que puede ser más de 0 segundos.

Es una forma de que una tarea pueda ceder el control por un momento para permitir que otras tareas se ejecuten.

La diferencia importante entre `asyncio.sleep()` y `time.sleep()` es que `asyncio.sleep()` es no bloqueante y `time.sleep()` es bloqueante:

- `time.sleep()`: bloquea el hilo actual. Mientras está bloqueado, otros hilos pueden ejecutarse. En la mayoría de ocasiones, esto supone bloquear el agente completo.
- `asyncio.sleep()`: bloquea la corutina actual (tarea `asyncio`). Mientras está bloqueada, otras corutinas pueden ejecutarse. Esto va a suponer, normalmente, bloquear el comportamiento actual del agente, permitiendo entrar otros comportamientos.

## Ejemplo 2: Un agente con un comportamiento - `dummyagent_behav_self_kill.py`

La programación de agentes SPADE se realiza en su mayoría mediante comportamientos (`behaviors`).

Vamos a crear un comportamiento cíclico que realiza una tarea (un contador simple). Para ello definimos un comportamiento llamado `MyBehav`que hereda de la clase `spade.behaviour.CyclicBehaviour`. Esta clase representa un comportamiento cíclico que no tiene un periodo especificado, es decir, un comportamiento tipo bucle. 

Hay dos métodos especiales en el comportamiento:

- `on_start()`: Este método es similar al método `setup()` de la clase agente. Se ejecuta antes de que comience la iteración principal del comportamiento y se usa para el código de inicialización. En este caso, imprimimos una línea e inicializamos la variable contador.
- `on_end()`: Se ejecuta cuando el comportamiento se ha realizado o eliminado.
- `run()`: Éste es el método donde se realiza el núcleo de la programación del comportamient, porque es el que se llama en cada iteración del ciclo del comportamiento. Actúa como el cuerpo del bucle. En nuestro ejemplo, imprime el valor actual del contador, lo incrementa y si dicho contador es mayor que 3, mata el comportamiento. En otro caso, espera por un segundo (para iterar de nuevo).

El método `run()` es una corutina asíncrona. Esto es muy importante, ya que SPADE es una biblioteca asíncrona basada en `asyncio` de python. Por eso podemos llamar a métodos asíncronos desde dentro del método `run()`, como `await asyncio.sleep(1)`, que permite dormir durante un segundo el comportamiento sin bloquear el bucle de eventos.

In [9]:
import asyncio
import spade
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

class DummyAgent(Agent):
    class MyBehav(CyclicBehaviour):
        async def on_start(self):
            print("Starting behaviour . . .")
            self.counter = 0

        async def run(self):
            print("Counter: {}".format(self.counter))
            self.counter += 1
            if self.counter > 3:
                self.kill(exit_code=10)
                return
            await asyncio.sleep(1)

        async def on_end(self):
            print("Behaviour finished with exit code {}.".format(self.exit_code))

    async def setup(self):
        print("Agent starting . . .")
        self.my_behav = self.MyBehav()
        self.add_behaviour(self.my_behav)

### Añadid vuestro login como parte del JID del agente
dummy = DummyAgent("Prueba_SPADE_brivaro@gtirouter.dsic.upv.es", "SPADE")
await dummy.start()

### wait until user interrupts with ctrl+C
while not dummy.my_behav.is_killed():
    try:
        await asyncio.sleep(1)
    except KeyboardInterrupt:
        break

assert dummy.my_behav.exit_code == 10

await dummy.stop()

Agent starting . . .
Starting behaviour . . .
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Behaviour finished with exit code 10.


## Ejemplo 3: Emisor y Receptor - `sender_receiver.py`

In [10]:
import spade
from spade.agent import Agent
from spade.behaviour import OneShotBehaviour
from spade.message import Message

class SenderAgent(Agent):
    class SendBehaviour(OneShotBehaviour):
        async def run(self):
            ### Añadid vuestro login como parte del JID del agente
            msg = Message(to="receiver_brivaro@gtirouter.dsic.upv.es")  # Destinatario
            msg.set_metadata("performative", "inform")  # Tipo de mensaje
            msg.body = "Hola desde el agente emisor! Eres un puto genio"

            await self.send(msg)
            print("Mensaje enviado!")

    async def setup(self):
        self.add_behaviour(self.SendBehaviour())

class ReceiverAgent(Agent):
    class ReceiveBehaviour(OneShotBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)  # Espera por un mensaje por 10 segundos
            if msg:
                print("Mensaje recibido:", msg.body)
            else:
                print("No se recibió ningún mensaje.")

    async def setup(self):
        self.add_behaviour(self.ReceiveBehaviour())

In [11]:
### Añadid vuestro login como parte del JID de los agentes
sender = SenderAgent("sender_brivaro@gtirouter.dsic.upv.es", "secret")
receiver = ReceiverAgent("receiver_brivaro@gtirouter.dsic.upv.es", "secret")

await receiver.start()

await sender.start()

Mensaje enviado!


Mensaje recibido: Hola desde el agente emisor! Eres un puto genio


## Ejemplo 4: Emisor periódico y Receptor que imprime los mensajes - `periodic_sender_receiver.py`

In [12]:
# Código para el sistema con dos agentes: uno envía mensajes periódicamente y el otro los recibe

import spade
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour
from spade.message import Message
import asyncio

class PeriodicSenderAgent(Agent):
    class SendPeriodicBehaviour(PeriodicBehaviour):
        async def run(self):
            ### Añadid vuestro login como parte del JID del agente
            msg = Message(to="receiver_brivaro@gtirouter.dsic.upv.es")  # Asegúrate de reemplazar esto con el JID correcto del receptor
            msg.set_metadata("performative", "inform")
            msg.body = "Mensaje periódico del agente emisor molongo"

            await self.send(msg)
            print("Mensaje enviado.")

    async def setup(self):
        # Ejemplo: enviar un mensaje cada 5 segundos
        periodic_behaviour = self.SendPeriodicBehaviour(period=5)
        self.add_behaviour(periodic_behaviour)

class ReceiverAgent(Agent):
    class ReceiveBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)  # Espera por un mensaje por 10 segundos
            if msg:
                print("Mensaje recibido:", msg.body)
            else:
                print("Tiempo de espera excedido sin recibir mensaje.")

    async def setup(self):
        self.add_behaviour(self.ReceiveBehaviour())

In [ ]:
### Añadid vuestro login como parte del JID de los agentes
sender = PeriodicSenderAgent("sender_brivaro@gtirouter.dsic.upv.es", "secret")
receiver = ReceiverAgent("receiver_brivaro@gtirouter.dsic.upv.es", "secret")
await receiver.start()
await sender.start()

| Ejercicio 1 |
|------------:|
| Modifica el ejemplo anterior para que 20 segundos después de haber comenzado la ejecución de los dos agentes, los elimine.|

In [ ]:
sender = PeriodicSenderAgent("sender_brivaro@gtirouter.dsic.upv.es", "secret")
receiver = ReceiverAgent("receiver_brivaro@gtirouter.dsic.upv.es", "secret")

await receiver.start()
await sender.start()

print("Los agentes han comenzado. Se detendrán en 20 segundos...")

await asyncio.sleep(20)  # Espera 20 segundos

print("Deteniendo los agentes...")
await sender.stop()
await receiver.stop()
print("Agentes detenidos.")

Mensaje recibido: Mensaje periódico del agente emisor molongo
Los agentes han comenzado. Se detendrán en 20 segundos...
Mensaje enviado.
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje enviado.
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje enviado.
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje enviado.
Mensaje recibido: Mensaje periódico del agente emisor molongo
Mensaje recibido: Mensaje periódico del agente emisor molongo
Deteniendo los agentes...
Agentes detenidos.


Mensaje recibido: Mensaje periódico del agente emisor molongo


## Ejemplo 5: Un agente propone a otros agentes que le envien un dato y simplemente lo almacena - `solicitar_dato_receptores.py`

In [16]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            ### Añadid vuestro login como parte del JID de los agentes
            receivers = ["agente_receptor51_brivaro@gtirouter.dsic.upv.es", "agente_receptor52_brivaro@gtirouter.dsic.upv.es", "agente_receptor53_brivaro@gtirouter.dsic.upv.es"]
            for receiver in receivers:
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            self.agent.responses = []
            for _ in range(3):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses.append(response.body)
            print("Respuestas recibidas:", self.agent.responses)

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")


In [17]:
# Crear agentes
### Añadid vuestro login como parte del JID de los agentes
agente_propositor = AgentePropositor("agente_propositor5_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("agente_receptor51_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("agente_receptor52_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("agente_receptor53_brivaro@gtirouter.dsic.upv.es", "secret")

# Iniciar agentes

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()

await agente_propositor.start()

agente_receptor51_brivaro iniciado
agente_receptor52_brivaro iniciado
agente_receptor53_brivaro iniciado
agente_propositor5_brivaro iniciado
Propuesta enviada a agente_receptor51_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor52_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor53_brivaro@gtirouter.dsic.upv.es


agente_receptor51_brivaro respondió con 34
agente_receptor52_brivaro respondió con 65
agente_receptor53_brivaro respondió con 58
Respuestas recibidas: ['34', '65', '58']


| Ejercicio 2 |
|------------:|
| Modifica el ejemplo anterior para que 20 segundos después de haber comenzado la ejecución de los agentes, los elimine.|

In [18]:
agente_propositor = AgentePropositor("agente_propositor5_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("agente_receptor51_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("agente_receptor52_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("agente_receptor53_brivaro@gtirouter.dsic.upv.es", "secret")

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()

await agente_propositor.start()

print("Los agentes han comenzado. Se detendrán en 20 segundos...")

await asyncio.sleep(20)  # Espera 20 segundos

print("Deteniendo los agentes...")
await agente_receptor1.stop()
await agente_receptor2.stop()
await agente_receptor3.stop()

await agente_propositor.stop()
print("Agentes detenidos.")

agente_receptor51_brivaro iniciado
agente_receptor52_brivaro iniciado
agente_receptor53_brivaro iniciado
agente_propositor5_brivaro iniciado
Los agentes han comenzado. Se detendrán en 20 segundos...
Propuesta enviada a agente_receptor51_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor52_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor53_brivaro@gtirouter.dsic.upv.es
agente_receptor51_brivaro respondió con 97
agente_receptor52_brivaro respondió con 88
agente_receptor53_brivaro respondió con 23
Respuestas recibidas: ['97', '88', '23']
Deteniendo los agentes...
Agentes detenidos.


``voy a añadir un end para que los agentes se despidan``

In [19]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            ### Añadid vuestro login como parte del JID de los agentes
            receivers = ["agente_receptor51_brivaro@gtirouter.dsic.upv.es", "agente_receptor52_brivaro@gtirouter.dsic.upv.es", "agente_receptor53_brivaro@gtirouter.dsic.upv.es"]
            for receiver in receivers:
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            self.agent.responses = []
            for _ in range(3):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses.append(response.body)
            print("Respuestas recibidas:", self.agent.responses)
        
        async def on_end(self):
            print("Me las piro, vampiro :) soy el agente con JID: ", self.agent.name)

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")
            else:
                print("No se recibió ningún mensaje en el tiempo especificado.")
        async def on_end(self):
            print("Me las piro, vampiro :) soy el agente con JID: ", self.agent.name)

In [20]:
agente_propositor = AgentePropositor("agente_propositor5_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("agente_receptor51_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("agente_receptor52_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("agente_receptor53_brivaro@gtirouter.dsic.upv.es", "secret")

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()

await agente_propositor.start()

print("Los agentes han comenzado. Se detendrán en 20 segundos...")

await asyncio.sleep(20)  # Espera 20 segundos

print("Deteniendo los agentes...")
await agente_receptor1.stop()
await agente_receptor2.stop()
await agente_receptor3.stop()

await agente_propositor.stop()
print("Agentes detenidos.")

agente_receptor51_brivaro iniciado
agente_receptor52_brivaro iniciado
agente_receptor53_brivaro iniciado
agente_propositor5_brivaro iniciado
Los agentes han comenzado. Se detendrán en 20 segundos...
Propuesta enviada a agente_receptor51_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor52_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a agente_receptor53_brivaro@gtirouter.dsic.upv.es
agente_receptor51_brivaro respondió con 83
agente_receptor52_brivaro respondió con 89
agente_receptor53_brivaro respondió con 63
Respuestas recibidas: ['83', '89', '63']
Me las piro, vampiro :) soy el agente con JID:  agente_propositor5_brivaro
No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
Deteniendo los a

No se recibió ningún mensaje en el tiempo especificado.
No se recibió ningún mensaje en el tiempo especificado.
Me las piro, vampiro :) soy el agente con JID:  agente_receptor52_brivaro
Me las piro, vampiro :) soy el agente con JID:  agente_receptor53_brivaro


## Ejemplo 6: Un agente propone a otros agentes, recibe respuestas, las procesa y selecciona una contestando al elegido que ha sido seleccionado - `solicitar_dato_elegir.py` 

In [21]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            ### Añadid vuestro login como parte del JID de los agentes
            receivers = ["receptor61_brivaro@gtirouter.dsic.upv.es", "receptor62_brivaro@gtirouter.dsic.upv.es", "receptor63_brivaro@gtirouter.dsic.upv.es"]
            self.agent.responses = {}

            # Enviar propuestas a cada receptor
            for receiver in receivers:
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            for _ in range(3):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses[str(response.sender).split("@")[0]] = response.body

            print("Respuestas recibidas:", self.agent.responses)

            # Seleccionar y enviar respuesta
            if self.agent.responses:
                selected_agent, selected_value = random.choice(list(self.agent.responses.items()))
                response_msg = spade.message.Message(
                    to=f"{selected_agent}@gtirouter.dsic.upv.es",
                    body=f"Tu valor {selected_value} fue seleccionado",
                    metadata={"performative": "inform"}
                )
                await self.send(response_msg)
                print(f"Enviada respuesta a {selected_agent} sobre valor seleccionado: {selected_value}")

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())
        self.add_behaviour(self.InformBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata["performative"] == "propose":
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")

    class InformBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata["performative"] == "inform":
                print(f"{self.agent.name} recibió mensaje de selección: {msg.body}")


In [22]:
# Crear agentes
### Añadid vuestro login como parte del JID de los agentes
agente_propositor = AgentePropositor("propositor6_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("receptor61_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("receptor62_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("receptor63_brivaro@gtirouter.dsic.upv.es", "secret")

# Iniciar agentes

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()
await asyncio.sleep(1)
await agente_propositor.start()

receptor61_brivaro iniciado
receptor62_brivaro iniciado
receptor63_brivaro iniciado
propositor6_brivaro iniciado
Propuesta enviada a receptor61_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor62_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor63_brivaro@gtirouter.dsic.upv.es


receptor61_brivaro respondió con 75
receptor62_brivaro respondió con 75
receptor63_brivaro respondió con 34
Respuestas recibidas: {'receptor61_brivaro': '75', 'receptor62_brivaro': '75', 'receptor63_brivaro': '34'}
Enviada respuesta a receptor63_brivaro sobre valor seleccionado: 34
receptor63_brivaro recibió mensaje de selección: Tu valor 34 fue seleccionado


| Ejercicio 3 |
|------------:|
| Modifica el ejemplo anterior para que 20 segundos después de haber comenzado la ejecución de los agentes, los elimine.|

In [23]:
agente_propositor = AgentePropositor("propositor6_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("receptor61_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("receptor62_brivaro@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("receptor63_brivaro@gtirouter.dsic.upv.es", "secret")

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()

await agente_propositor.start()

print("Los agentes han comenzado. Se detendrán en 20 segundos...")

await asyncio.sleep(20)  # Espera 20 segundos

print("Deteniendo los agentes...")
await agente_receptor1.stop()
await agente_receptor2.stop()
await agente_receptor3.stop()

await agente_propositor.stop()
print("Agentes detenidos.")

receptor61_brivaro iniciado
receptor62_brivaro iniciado
receptor63_brivaro iniciado
propositor6_brivaro iniciado
Los agentes han comenzado. Se detendrán en 20 segundos...
Propuesta enviada a receptor61_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor62_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor63_brivaro@gtirouter.dsic.upv.es
receptor61_brivaro respondió con 96
receptor62_brivaro respondió con 31
receptor63_brivaro respondió con 61
Respuestas recibidas: {'receptor61_brivaro': '96', 'receptor62_brivaro': '31', 'receptor63_brivaro': '61'}
Enviada respuesta a receptor61_brivaro sobre valor seleccionado: 96
receptor61_brivaro recibió mensaje de selección: Tu valor 96 fue seleccionado
Deteniendo los agentes...
Agentes detenidos.


| Ejercicio 4 |
|------------:|
| Generaliza el ejemplo anterior para que el sistema esté compuesto por un agente que propone y `n` que responden. Para ello, generar una función `crear_ags_receptores` que reciba como argumento el número de agentes a crear, que deberá haberse solicitado al usuario y que cree dichos agentes y acabe devolviendo una lista formada por las instancias de dichos agentes que ha creado.|

In [24]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            # Obtener los JIDs de los agentes receptores a partir de las instancias creadas
            # Se asume que cada agente receptor tiene su jid en el formato: "receptorX_brivaro@gtirouter.dsic.upv.es"
            # Guardamos las respuestas en un diccionario
            self.agent.responses = {}

            # Enviar propuestas a cada receptor (recorremos la lista de receptores ya iniciada en self.agent.receptores)
            for receptor in self.agent.receptores:
                msg = spade.message.Message(
                    to=str(receptor.jid),
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receptor.jid}")

            # Esperar y recoger respuestas de todos los receptores
            for _ in range(len(self.agent.receptores)):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses[str(response.sender).split("@")[0]] = response.body

            print("Respuestas recibidas:", self.agent.responses)

            # Seleccionar aleatoriamente una respuesta y enviar la confirmación
            if self.agent.responses:
                selected_agent, selected_value = random.choice(list(self.agent.responses.items()))
                response_msg = spade.message.Message(
                    to=f"{selected_agent}@gtirouter.dsic.upv.es",
                    body=f"Tu valor {selected_value} fue seleccionado",
                    metadata={"performative": "inform"}
                )
                await self.send(response_msg)
                print(f"Enviada respuesta a {selected_agent}@gtirouter.dsic.upv.es sobre valor seleccionado: {selected_value}")

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())
        self.add_behaviour(self.InformBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata.get("performative") == "propose":
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")

    class InformBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata.get("performative") == "inform":
                print(f"{self.agent.name} recibió mensaje de selección: {msg.body}")

# Función para crear 'n' agentes receptores
def crear_ags_receptores(n):
    agentes = []
    for i in range(1, n+1):
        jid = f"receptor{i}_brivaro@gtirouter.dsic.upv.es"
        agente = AgenteReceptor(jid, "secret")
        agentes.append(agente)
    return agentes


In [27]:
# Solicitar al usuario el número de agentes receptores
n = int(input("Ingrese el número de agentes receptores a crear: "))
receptores = crear_ags_receptores(n)

# Crear el agente propositor
agente_propositor = AgentePropositor("propositor7_brivaro@gtirouter.dsic.upv.es", "secret")

# Iniciar todos los agentes receptores
for agente in receptores:
    await agente.start()

# Asignar la lista de receptores al agente propositor para que conozca los JIDs
agente_propositor.receptores = receptores
await agente_propositor.start()

print("Los agentes han comenzado. Se detendrán en 20 segundos...")
await asyncio.sleep(20)  # Espera 20 segundos

print("Deteniendo los agentes...")
for agente in receptores:
    await agente.stop()
await agente_propositor.stop()
print("Agentes detenidos.")

receptor1_brivaro iniciado
receptor2_brivaro iniciado
receptor3_brivaro iniciado
receptor4_brivaro iniciado
receptor5_brivaro iniciado
propositor7_brivaro iniciado
Los agentes han comenzado. Se detendrán en 20 segundos...
Propuesta enviada a receptor1_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor2_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor3_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor4_brivaro@gtirouter.dsic.upv.es
Propuesta enviada a receptor5_brivaro@gtirouter.dsic.upv.es
receptor1_brivaro respondió con 30
receptor2_brivaro respondió con 47
receptor3_brivaro respondió con 55
receptor4_brivaro respondió con 97
receptor5_brivaro respondió con 6
Respuestas recibidas: {'receptor1_brivaro': '30', 'receptor2_brivaro': '47', 'receptor3_brivaro': '55', 'receptor4_brivaro': '97', 'receptor5_brivaro': '6'}
Enviada respuesta a receptor4_brivaro@gtirouter.dsic.upv.es sobre valor seleccionado: 97
receptor4_brivaro recibió mensaje de selección: Tu valor 9

# [SPADE-BDI](https://pypi.org/project/spade-bdi/) 

Plugin de **SPADE** que permite incluir un comportamiento BDI con sintaxis **AgentSpeak**.

## Instalar SPADE BDI

In [28]:
try:
    import spade_bdi
except:
    !pip install spade_bdi

### Ejemplo 1: un agente SPADE - BDI básico que carga un fichero ASL con un conocimiento en AgentSpeak - `basic_bdi.py`

Creamos el fichero `basic.asl`: 

In [29]:
with open('basic.asl', 'w') as fp:
    fp.write('!start.\n')
    fp.write('+!start <-\n')
    fp.write('    +car(rojo);\n')
    fp.write('    +truck(azul).\n')

    fp.write('+car(Color)\n') 
    fp.write('    <- .print("El carro es ",Color).!start.\n')

In [30]:
!more basic.asl

!start.
+!start <-
    +car(rojo);
    +truck(azul).
+car(Color)
    <- .print("El carro es ",Color).!start.


Creamos el agente y lo ejecutamos (cargando `basic.asl` como conocimiento deliberativo del agente):

In [31]:
import asyncio

import time

import spade

from spade_bdi.bdi import BDIAgent

a = BDIAgent("BasicAgent_BDI_brivaro@gtirouter.dsic.upv.es", "SPADE", "basic.asl")

await a.start()

await asyncio.sleep(1)

print("Comenzando a interactuar con el comportamiento BDI del agente ... ");
a.bdi.set_belief("car", "azul", "big")
a.bdi.print_beliefs()
print("GETTING FIRST CAR BELIEF")
print("Primera Creencia de tipo car: ", a.bdi.get_belief("car"))
a.bdi.print_beliefs()
a.bdi.remove_belief("car", 'azul', "big")
a.bdi.print_beliefs()
print(a.bdi.get_beliefs())
a.bdi.set_belief("car", 'amarillo')

time.sleep(1)

await a.stop()   

basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
Comenzando a interactuar con el comportamiento BDI del agente ... 
car(rojo)
truck(azul)
GETTING FIRST CAR BELIEF
Primera Creencia de tipo car:  car(rojo)
car(rojo)
truck(azul)
car(rojo)
truck(azul)
['car(rojo)', 'truck(azul)']


Comprobad la diferencia en la ejecución con el cambio que se muestra a continuación:

In [32]:
import asyncio

import time

import spade

from spade_bdi.bdi import BDIAgent

a = BDIAgent("BasicAgent_BDI_brivaro@gtirouter.dsic.upv.es", "SPADE", "basic.asl")

await a.start()

await asyncio.sleep(1)

print("Comenzando a interactuar con el comportamiento BDI del agente ... ");
a.bdi.set_belief("car", "azul", "big")

await asyncio.sleep(1)    ### CAMBIO: Dejamos un poco de tiempo para actualizar el comportamiento BDI

a.bdi.print_beliefs()
print("GETTING FIRST CAR BELIEF")
print("Primera Creencia de tipo car: ", a.bdi.get_belief("car"))
a.bdi.print_beliefs()
a.bdi.remove_belief("car", 'azul', "big")
a.bdi.print_beliefs()
print(a.bdi.get_beliefs())
a.bdi.set_belief("car", 'amarillo')

time.sleep(1)

await a.stop()   

basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
Comenzando a interactuar con el comportamiento BDI del agente ... 
car(rojo)
truck(azul)
car(azul, big)
GETTING FIRST CAR BELIEF
Primera Creencia de tipo car:  car(rojo)
car(rojo)
truck(azul)
car(azul, big)
car(rojo)
truck(azul)
car(azul, big)
['car(rojo)', 'truck(azul)', 'car(azul, big)']


In [33]:
a = BDIAgent("BasicAgent_BDI_brivaro@gtirouter.dsic.upv.es", "SPADE", "basic.asl")

await a.start()

await asyncio.sleep(1)

print("Comenzando a interactuar con el comportamiento BDI del agente ... ");
a.bdi.set_belief("car", "azul", "big")

await asyncio.sleep(2)    ### CAMBIO: Dejamos un poco de tiempo para actualizar el comportamiento BDI

a.bdi.print_beliefs()

await asyncio.sleep(2) 

print("GETTING FIRST CAR BELIEF")
print("Primera Creencia de tipo car: ", a.bdi.get_belief("car"))

await asyncio.sleep(2) 

a.bdi.print_beliefs()
a.bdi.remove_belief("car", 'azul', "big")
a.bdi.print_beliefs()
print(a.bdi.get_beliefs())
a.bdi.set_belief("car", 'amarillo')

time.sleep(1)

await a.stop()   

basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
basicagent_bdi_brivaro@gtirouter.dsic.upv.es El carro es  rojo
Comenzando a interactuar con el comportamiento BDI del agente ... 
car(rojo)
truck(azul)
car(azul, big)
GETTING FIRST CAR BELIEF
Primera Creencia de tipo car:  car(rojo)
car(rojo)
truck(azul)
car(azul, big)
car(rojo)
truck(azul)
car(azul, big)
['car(rojo)', 'truck(azul)', 'car(azul, big)']


### Ejemplo 2: Creación de acciones internas en un agente SPADE - BDI - `actions.py`

Creamos el fichero `actions.asl`: 

In [34]:
with open('actions.asl', 'w') as fp:
    fp.write('!start.\n')
    fp.write('+!start <-\n')
    fp.write('    .my_function(4, R);\n')
    fp.write('    .my_action(R).\n')

In [35]:
!more actions.asl

!start.
+!start <-
    .my_function(4, R);
    .my_action(R).


In [36]:
import asyncio
import argparse

import agentspeak

import spade

from spade_bdi.bdi import BDIAgent


class MyCustomBDIAgent(BDIAgent):
    def add_custom_actions(self, actions):
        
        @actions.add_function(".my_function", (int,))
        def _my_function(x):
            return x * x

        @actions.add(".my_action", 1)
        def _my_action(agent, term, intention):
            arg = agentspeak.grounded(term.args[0], intention.scope)
            print(arg)
            yield

In [37]:
a = MyCustomBDIAgent("bdi_agent_actions_brivaro@gtirouter.dsic.upv.es", "bdipassword", "actions.asl")

await a.start()

await asyncio.sleep(2)

await a.stop()

16


### Ejemplo 3: Creación de un agente con un comportamiento BDI, que reciba como parámetro el nombre del fichero con el código BDI (fichero `.asl`) y el tiempo de ejecución de dicho agente. - `asl_launcher.py`

In [38]:
import asyncio
import argparse

import spade

from spade_bdi.bdi import BDIAgent

async def create_par_bdi_agent( sAsl_name, nTime ):

	a = MyCustomBDIAgent("Agent_par_BDI_brivaro@gtirouter.dsic.upv.es", "bdi_agent", sAsl_name)

	await a.start()

	#version web
	a.web.start(hostname="127.0.0.1",port="10000")

	await asyncio.sleep(nTime)

	await a.stop()


Creamos el fichero `actions.asl`: 

In [39]:
with open('actions.asl', 'w') as fp:
    fp.write('!start.\n')
    fp.write('+!start <-\n')
    fp.write('    .my_function(4, R);\n')
    fp.write('    .my_action(R).\n')

In [40]:
!more actions.asl

!start.
+!start <-
    .my_function(4, R);
    .my_action(R).


In [41]:
await create_par_bdi_agent( "actions.asl", 2 )

16


Podéis conectar a la [interfaz web del agente](http://127.0.0.1:10000/spade). 

| Ejercicio 5 |
|------------:|
| Usando la función `create_par_bdi_agent()`, crea un asl y ejecútalo aquí para poder calcular un factorial de cualquier número dado por el usuario.|

In [42]:
import asyncio
import agentspeak
import spade
from spade_bdi.bdi import BDIAgent

# Función auxiliar para calcular el factorial
def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)

# Clase personalizada que registra las acciones custom necesarias
class FactorialBDIAgent(BDIAgent):
    def add_custom_actions(self, actions):
        @actions.add_function(".fact", (int,))
        def _fact(n):
            return factorial(n)
        
        @actions.add(".print_result", 2)
        def _print_result(agent, term, intention):
            num = agentspeak.grounded(term.args[0], intention.scope)
            res = agentspeak.grounded(term.args[1], intention.scope)
            print(f"El factorial de {num} es {res}")
            yield

num_str = input("Ingresa el número para calcular el factorial: ")
num = int(num_str)
with open("factorial.asl", "w") as fp:
    fp.write("!start.\n")
    fp.write(f"+!start <- .fact({num}, R);\n")
    fp.write(f"    .print_result({num}, R).\n")

async def create_par_bdi_agent(asl_file, duration):
    a = FactorialBDIAgent("Agent_factorial_brivaro@gtirouter.dsic.upv.es", "bdi_agent", asl_file)
    await a.start()
    a.web.start(hostname="127.0.0.1", port="10000")
    await asyncio.sleep(duration)
    await a.stop()

await create_par_bdi_agent("factorial.asl", 10)


El factorial de 8.0 es 40320


Nos conectamos y lo matamos en [interfaz web del agente](http://127.0.0.1:10000/spade), pero a los 10 segundos se irá a descansar ;).

# BOWLDI

The **BOWLDI** idea is to combine ontologies (OWL) and BDI agents. In particular, it is envisioned to work with the **SPADE** platform, i.e. the BDI implementation used by SPADE, which is based on the AgentSpeak language. Therefore, BOWLDI allows to use OWL ontologies in the BDI agents implemented in SPADE. This is achieved by translating OWL/RDF files into AgentSpeak code and vice versa.

The conversion is performed within the `BOWLDIConverter` Python class that uses `owlready2` library to read ontology files or store information into them. Inversely, the said class can also convert AgentSpeak code into OWL/RDF files. The conversion is based on the mapping of OWL classes and properties to AgentSpeak plans and beliefs.

The class is envisioned as a modular construct that supports various functionalities, each of which may be used on their own as well. The main functionalities are:
- **Ontology to AgentSpeak conversion**: The conversion of OWL ontologies into AgentSpeak code.
- **AgentSpeak to Ontology conversion**: The conversion of AgentSpeak code into OWL ontologies.

Using the `BOWLDIConverter` class can be as easy as creating an instance of the class and calling the desired method.

The implementation of the `BOWLDIConverter` class is publicly available on GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Extra/BOWLDI/bowldi.py).

You can download it e.g. by using wget.

In [1]:
!pip install wget

In [2]:
!wget https://raw.githubusercontent.com/AILab-FOI/MAGO/main/Extra/BOWLDI/bowldi.py

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


The only library necessary for `BOWLDIConverter` to work is `owlready2`. You can install it by running the following command:

In [3]:
try:
    import owlready2
except:
    !pip install owlready2

Otherwise, you can create a new Python environment with the necessary libraries. One possible solution is to create a Conda environment using the following environment definition:

```yaml
name: bowldi

dependencies:
  - python=3.10
  - conda-forge::owlready2
  - conda-forge::pexpect
  - pip
  - pip:
    - spade
    - spade_bdi

# conda env create -f env.yml
# conda env update -f env.yml --prune
```

This file is also available publicly at GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/env.yml), and can be downloaded using wget as well:

```bash
wget https://raw.githubusercontent.com/AILab-FOI/MAGO/main/Documents/250320%20Class/env.yml
```

## Example 1: Converting AgentSpeak to OWL

`BOWLDIConverter` can take a string input of AgentSpeak code and convert it to an ontology file. The following is a simple example containing: a couple of beliefs. These can be mapped to:
- two ontology concepts (lines 5--7),
- one individual (line 10),
- two properties (lines 13 and 16).

In [4]:
from bowldi import BOWLDIConverter

input_data = """
// Concepts and Hierarchies
concept(person).
concept(wizard).
is_a(wizard, person).

// Individual
individual(gandalf, wizard).

// Object Property
object_property(person, is_friend_with, person).

// Data Property
data_property(person, has name, string)."""

converter = BOWLDIConverter(
    input_data=input_data,
)

converter.get_response()

Conversion complete.
File already exists. Overwriting.
Output saved to w:\SPADE\output.owl


{'status': 'success',
 'message': 'Conversion complete.',
 'file': 'w:/SPADE/output.owl'}

In [5]:
!more output.owl

<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xml:base="file:///w:/SPADE/output.owl"
         xmlns="file:///w:/SPADE/output.owl#">

<owl:Ontology rdf:about="file:///w:/SPADE/output.owl"/>

<owl:ObjectProperty rdf:about="#is_friend_with">
  <rdfs:domain rdf:resource="#person"/>
  <rdfs:range rdf:resource="#person"/>
  <rdfs:label xml:lang="en-gb">is friend with</rdfs:label>
  <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
</owl:ObjectProperty>

<owl:DatatypeProperty rdf:about="#has_name">
  <rdfs:domain rdf:resource="#person"/>
  <rdfs:range rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
  <rdfs:label xml:lang="en-gb">has name</rdfs:label>
  <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">

The response received on line 22 should read:

```json
{
  "status": "success",
  "message": "Conversion complete.",
  "file": "<path to file>/output.owl"
}
```

Converting AgentSpeak to an ontology file will always yield a file output, i.e. will always write the output to a file. The generated file `output.owl` should have the following content, based on the input AgentSpeak code:

```xml
<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
    xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
    xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
    xmlns:owl="http://www.w3.org/2002/07/owl#" xml:base="file://<path to file>/output.owl"
    xmlns="file://<path to file>/output.owl#">

    <owl:Ontology rdf:about="file://<path to file>/output.owl"/>

    <owl:ObjectProperty rdf:about="#is_friend_with">
        <rdfs:domain rdf:resource="#person"/>
        <rdfs:range rdf:resource="#person"/>
        <rdfs:label xml:lang="en-gb">is friend with</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:ObjectProperty>

    <owl:DatatypeProperty rdf:about="#has_name">
        <rdfs:domain rdf:resource="#person"/>
        <rdfs:range rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
        <rdfs:label xml:lang="en-gb">has name</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:DatatypeProperty>

    <owl:Class rdf:about="#person">
        <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
        <rdfs:label xml:lang="en-gb">person</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:Class>

    <owl:Class rdf:about="#wizard">
        <rdfs:subClassOf rdf:resource="#person"/>
        <rdfs:label xml:lang="en-gb">wizard</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:Class>

    <owl:NamedIndividual rdf:about="#gandalf">
        <rdf:type rdf:resource="#wizard"/>
        <rdfs:label xml:lang="en-gb">gandalf</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:NamedIndividual>

</rdf:RDF>
```

## Example 2: Converting OWL to AgentSpeak

`BOWLDIConverter` class can take an ontology file and convert it to AgentSpeak code. The following is a simple example of reading the ontology created in the previous example and converting it to AgentSpeak code.

In [6]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="output.owl",
    output_data_path="example_output.asl",
)

if converter.get_response().get("status") == "success":
    print(converter.agentspeak_output)

Conversion complete. Output saved to w:\SPADE\example_output.asl



In [7]:
!more example_output.asl

The above code will take the ontology stored in the file `output.owl` and convert it to AgentSpeak code. The output is saved in the provided `output_data_path` file. In addition to receiving the output saved into the given file, the output can also be retrieved as a string using the `agentspeak_output` attribute.

The rendered AgentSpeak code should look as follows, based on the expected input:

```prolog
concept(person)[source(['self'])].
concept(wizard)[source(['self'])].
is_a(wizard, person)[source(['self'])].
object_property(person, is_friend_with, person)[source(['self'])].
data_property(person, has_name, str)[source(['self'])].
individual(gandalf, wizard)[source(['self'])].
```

This last output features a list of beliefs, but this time each of those is given a `source` attribute. This attribute should be used to tell the agent where the particular piece of knowledge comes from, i.e. from which agent or knowledge source (e.g. an ontology) it originates. In this instance, all the knowledge comes from the agent itself, hence the `self` value.

## Example 3: Converting OWL to AgentSpeak with an external source

A simple ontology has been prepared, featuring a couple of concepts and properties, related to the domain of the Lord of the Rings novel. The ontology is available publicly on GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/lotr_example.owl), and it can be visualized using WebOWL in this [link](https://service.tib.eu/webvowl/#iri=https://raw.githubusercontent.com/AILab-FOI/MAGO/refs/heads/main/Documents/250320%20Class/onto_example.rdf#)

The linked ontology contains the following information:

**Classes**

- `person`
- `wizard` (subclass of `person`)
- `elf` (subclass of `person`)
- `human` (subclass of `person`)
- `dwarf` (subclass of `person`)
- `hobbit` (subclass of `person`)
- `kingdom`
- `ring_of_power`

**Object Properties**

- `has_king` (domain: `kingdom`, range: `person`)
- `is_friend_with` (domain: `person`, range: `person`)
- `is_in_team_with` (domain: `person`, range: `person`)
- `has_ring` (domain: `person`, range: `ring_of_power`)
- `fights_against` (domain: `person`, range: `person`)

**Datatype Properties**

- `has_name` (domain: `person`, range: `string`)
- `description` (domain: `ring_of_power`, range: `string`)

Importing the above ontology, let us prepare another ontology that uses the concepts of the imported ontology to construct additional data. The developed example ontology that imports the `lotr_ontology.owl` is available [online](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/onto_example.rdf) as well. It may be downloaded e.g. by using `wget`.

In [52]:
!wget https://raw.githubusercontent.com/AILab-FOI/MAGO/refs/heads/main/Documents/250320%20Class/onto_example.rdf

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


This example ontology extends the `lotr_ontology.owl` and defines individuals of the classes stored therein.

The three defined individuals are described as follows, using the RDF/XML syntax generated by Protégé:

```xml
    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779 -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#wizard"/>
        <lotr:has_name>Gandalf the Grey</lotr:has_name>
        <lotr:has_name>Mithrandir</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Gandalf the Grey</rdfs:label>
    </owl:NamedIndividual>
    <owl:Axiom>
        <owl:annotatedSource rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779"/>
        <owl:annotatedProperty rdf:resource="<path to onto>/lotr_example.owl#has_name"/>
        <owl:annotatedTarget>Gandalf the Grey</owl:annotatedTarget>
        <rdfs:isDefinedBy>Bogdan</rdfs:isDefinedBy>
    </owl:Axiom>
    


    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_8584c614_acbd_4080_9032_a3a6dd9d1a9b -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_8584c614_acbd_4080_9032_a3a6dd9d1a9b">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#human"/>
        <lotr:is_friend_with rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779"/>
        <lotr:is_friend_with rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4"/>
        <lotr:has_name>Aragorn, son of Arathorn</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Aragorn, son of Arathorn</rdfs:label>
    </owl:NamedIndividual>
    


    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4 -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#elf"/>
        <lotr:has_name>Legolas Greenleaf</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Legolas</rdfs:label>
    </owl:NamedIndividual>
```

When we use the `BOWLDIConvert` class to convert the information of `onto_example.rdf` into AgentSpeak code, the output includes a defined source value for every belief. Information that was defined in the `onto_example.rdf` ontology will be designated as `self`, while the information imported from the `lotr_example.owl` ontology will be marked using the imported ontology's IRI. The latter makes it possible to import the relevant ontology when converting the AgentSpeak code back to an ontology. Since we want the information that is transferred into AgentSpeak code to be complete, the various concepts of the imported ontology are included in the output.

In [53]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="onto_example.rdf",
)

if converter.get_response().get("status") == "success":
    print(converter.agentspeak_output)
else:
    print(converter.get_response())

{'status': 'error', 'message': 'Input file does not exist.'}


If everything was as expected, the AgentSpeak output should be saved in the `output.asl` file and look like follows:

```prolog
concept(wizard)[source(['<path to ontology>/lotr_example.owl'])].
is_a(wizard, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(human)[source(['<path to ontology>/lotr_example.owl'])].
is_a(human, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(elf)[source(['<path to ontology>/lotr_example.owl'])].
is_a(elf, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(kingdom)[source(['<path to ontology>/lotr_example.owl'])].
concept(person)[source(['<path to ontology>/lotr_example.owl'])].
concept(ring)[source(['<path to ontology>/lotr_example.owl'])].
concept(dwarf)[source(['<path to ontology>/lotr_example.owl'])].
is_a(dwarf, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(hobbit)[source(['<path to ontology>/lotr_example.owl'])].
is_a(hobbit, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, is_friend_with, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(kingdom, has_king, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, is_in_team_with, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, has_ring, ring)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, fights_against, person)[source(['<path to ontology>/lotr_example.owl'])].
data_property(person, has_name, str)[source(['<path to ontology>/lotr_example.owl'])].
data_property(ring, description, str)[source(['<path to ontology>/lotr_example.owl'])].
individual(gandalf_the_grey, wizard)[source(['self'])].
individual(aragorn_son_of_arathorn, human)[source(['self'])].
individual(legolas, elf)[source(['self'])].
relation(gandalf_the_grey, has_name, "Gandalf the Grey")[source(Bogdan)].
relation(gandalf_the_grey, has_name, "Mithrandir")[source(self)].
relation(aragorn_son_of_arathorn, is_friend_with, gandalf_the_grey)[source(self)].
relation(aragorn_son_of_arathorn, is_friend_with, legolas)[source(self)].
relation(aragorn_son_of_arathorn, has_name, "Aragorn, son of Arathorn")[source(self)].
relation(legolas, has_name, "Legolas Greenleaf")[source(self)].
```

## Example 4: Converting AgentSpeak with external source to OWL

The `BOWLDIConverter` class can convert AgentSpeak code into an ontology file even when source argument of some beliefs references another source. If that other source is an ontology (i.e. if the source is a link it is assumed to be an ontology), then the translation process will try to import the ontology before processing the rest of the contents of the AgentSpeak code. The following is a simple example of reading the AgentSpeak code created in the previous example and converting it to an ontology file.

In [54]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="output.asl",
)

print(converter.get_response())

{'status': 'error', 'message': 'Input file does not exist.'}


The output of the last line should be indicating success:

```json
{
  "status": "success",
  "message": "Conversion complete.",
  "file": "<path to file>/output.owl"
}
```

The rendered file should not contain definitions of the concepts defined in the imported ontology. Instead, only the concepts designated as not being sourced by an external ontology should be included.

One major difference between the initial ontolgoy of Example 3 above, and the output ontology of this example is that all the concepts that are "native" to the observed ontology (and not to the imported ontology) have one new property added, namely the `isDefinedBy` annotation property. The value of this annotation data property is sourced in the `source` argument of a specific belief. This property is used to indicate the source of the information, i.e. the agent that provided the information.